In [94]:
from pandas_profiling import ProfileReport
import pandas as pd

In [1333]:
df_accidents = pd.read_csv('../Road Safety Data/dft-road-casualty-statistics-accident-2020.csv')
df_casualties = pd.read_csv('../Road Safety Data/dft-road-casualty-statistics-casualty-2020.csv')
df_vehicles = pd.read_csv('../Road Safety Data/dft-road-casualty-statistics-vehicle-2020.csv')

C:\Python38\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning:

Columns (0,2) have mixed types.Specify dtype option on import or set low_memory=False.



In [1334]:
# accidents = ProfileReport(df_accidents)
# accidents.to_file(output_file='Data reports/accidents.html')
# casualties = ProfileReport(df_casualties)
# casualties.to_file(output_file='Data reports/casualties.html')
# vehicles = ProfileReport(df_vehicles)
# vehicles.to_file(output_file='Data reports/vehicles.html')

In [1335]:
casualties = df_casualties[['accident_reference', 'casualty_severity', 'vehicle_reference', 'casualty_reference', 'sex_of_casualty', 'age_band_of_casualty', 'casualty_home_area_type']]

In [1336]:
accidents = df_accidents[['accident_reference']]

In [1337]:
vehicles = df_vehicles[['accident_reference', 'vehicle_reference', 'vehicle_type', 'journey_purpose_of_driver', 'engine_capacity_cc', 'propulsion_code', 'age_of_vehicle', 'generic_make_model']]

In [1338]:
casualties

,accident_reference,casualty_severity,vehicle_reference,casualty_reference,sex_of_casualty,age_band_of_casualty,casualty_home_area_type
0,010219808,3,1,1,1,6,1
1,010220496,3,1,1,2,1,1
2,010220496,3,1,2,2,1,1
3,010228005,3,1,1,1,5,1
4,010228006,2,1,1,1,8,1
...,...,...,...,...,...,...,...
115579,991027064,2,2,1,1,3,1
115580,991029573,3,1,1,2,9,1
115581,991030297,2,2,1,1,7,2
115582,991030900,3,2,1,1,11,1


In [1339]:
c_and_a = pd.merge(casualties, accidents, on=['accident_reference'])

In [1340]:
all_data = pd.merge(c_and_a, vehicles, on=['accident_reference', 'vehicle_reference'])

In [1341]:
all_data = all_data.dropna()

In [1342]:
all_data = all_data[(all_data['sex_of_casualty'] != -1) & (all_data['sex_of_casualty'] != 9)] #delete missing

In [1343]:
all_data = all_data[all_data['age_band_of_casualty'] != -1] #delete missing
all_data = all_data[~all_data['age_band_of_casualty'].isin([1, 2, 3])]

In [1344]:
all_data = all_data[all_data['casualty_home_area_type'] != -1] #delete missing

In [1345]:
all_data = all_data[all_data['vehicle_type'] != -1] #delete missing
all_data = all_data[all_data['vehicle_type'].isin([9, 19, 3, 4, 5])]

In [1346]:
all_data = all_data[(all_data['journey_purpose_of_driver'] != -1)] #delete missing

In [1347]:
all_data = all_data[(all_data['generic_make_model'] != '-1')] #delete missing
all_data['generic_make_model'] = all_data['generic_make_model'].apply(lambda x: x.split(' ')[0])

In [1348]:
top_brands = all_data['generic_make_model'].value_counts().head(20).index
for i in range(len(all_data)):
    if all_data['generic_make_model'].iloc[i] not in top_brands:
        all_data['generic_make_model'].iloc[i] = 'Other'
        



C:\Python38\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [1349]:
vehicle_means = all_data.groupby('vehicle_type')['engine_capacity_cc'].mean()

In [1350]:
for i in range(len(all_data)):
    if all_data['engine_capacity_cc'].iloc[i] == -1:
        all_data['engine_capacity_cc'].iloc[i] = vehicle_means[all_data.iloc[i].vehicle_type]
    

In [1351]:
all_data = all_data[all_data['engine_capacity_cc'] != -1] #dropping rows/vehicles that have no engine capacity e.g. horse, cycle

In [1352]:
all_data = all_data[all_data['propulsion_code'] != -1] #dropping missing
all_data = all_data[all_data['propulsion_code'].isin([1, 2, 3, 8])]

In [1353]:
all_data = all_data[all_data['age_of_vehicle'] != -1] #dropping missing

In [1354]:
severity_dict = {
    1: 'Fatal',
    2: 'Serious',
    3: 'Slight'
}

severity_dict_reverse = {
    1: 3,
    2: 2,
    3: 1
}

all_data['casualty_severity_name'] = all_data['casualty_severity'].replace(severity_dict)
all_data['casualty_severity'] = all_data['casualty_severity'].replace(severity_dict_reverse)

In [1355]:
sex_dict = {
    1: 'Male',
    2: 'Female'
}

all_data['sex_of_casualty'] = all_data['sex_of_casualty'].replace(sex_dict)

In [1356]:
age_dict = {
    1:	'0 - 5',
    2: '6 - 10',
    3: '11 - 15',
    4: '16 - 20',
    5: '21 - 25',
    6: '26 - 35',
    7: '36 - 45',
    8: '46 - 55',
    9: '56 - 65',
    10: '66 - 75',
    11: '75+',
}

all_data['age_band_of_casualty'] = all_data['age_band_of_casualty'].replace(age_dict)

In [1357]:
home_dict = {
    1: 'Urban area', 
    2: 'Small town', 
    3: 'Rural', 
}

all_data['casualty_home_area_type'] = all_data['casualty_home_area_type'].replace(home_dict)

In [1358]:
vehicle_dict = { #changed names to group e.g. motorcycle
    1: 'Pedal cycle', 
    2: 'Motorcycle', 
    3: 'Motorcycle', 
    4: 'Motorcycle', 
    5: 'Motorcycle', 
    8: 'Taxi/Private hire car', 
    9: 'Car', 
    10: 'Minibus (8 - 16 passenger seats)', 
    11: 'Bus or coach (17 or more pass seats)', 
    16: 'Ridden horse', 
    17: 'Agricultural vehicle', 
    18: 'Tram', 
    19: 'Van', 
    20: 'Goods over 3.5t. and under 7.5t', 
    21: 'Goods 7.5 tonnes mgw and over', 
    22: 'Mobility scooter', 
    23: 'Electric motorcycle', 
    90: 'Other vehicle', 
    97: 'Motorcycle - unknown cc', 
    98: 'Goods vehicle - unknown weight', 
    99: 'Unknown vehicle type (self rep only)', 
    103: 'Motorcycle', 
    104: 'Motorcycle', 
    105: 'Motorcycle', 
    106: 'Motorcycle', 
    108: 'Taxi (excluding private hire cars) (1979-2004)', 
    109: 'Car (including private hire cars) (1979-2004)', 
    110: 'Minibus/Motor caravan (1979-1998)', 
    113: 'Goods over 3.5 tonnes (1979-1998)', 
}

all_data['vehicle_type'] = all_data['vehicle_type'].replace(vehicle_dict)

In [1359]:
journey_dict = { #changed names to group commuting styles
    1: 'Work',
    2: 'Work',
    3: 'School',
    4: 'School',
    5: 'Normal',
    6: 'Normal',
}

all_data['journey_purpose_of_driver'] = all_data['journey_purpose_of_driver'].replace(journey_dict)

In [1360]:
propulsion_dict = {
    1: 'Petrol',
    2: 'Heavy oil',
    3: 'Hybrid / Electric', #originally Electric
    4: 'Steam',
    5: 'Gas',
    6: 'Petrol/Gas (LPG)',
    7: 'Gas/Bi-fuel',
    8: 'Hybrid / Electric', #originally Hybrid Electric
    9: 'Gas Diesel',
    10: 'New fuel technology',
    11: 'Fuel cells',
    12: 'Electric diesel',
}

all_data['propulsion_code'] = all_data['propulsion_code'].replace(propulsion_dict)

In [1361]:
all_data['age_of_vehicle'] = all_data.groupby(['vehicle_type'])['age_of_vehicle'].transform(lambda x: pd.qcut(x, q=3, precision=0, duplicates='raise', labels=['new', 'middle-age', 'old']))

In [1362]:
# all_data['age_of_vehicle'] = all_data['age_of_vehicle'].astype('str').str.replace('-1.0', '0.0')

In [1363]:
all_data['engine_capacity_cc'] = all_data.groupby(['vehicle_type'])['engine_capacity_cc'].transform(lambda x: pd.cut(x, bins=3, precision=0, duplicates='raise', labels=['low', 'medium', 'high']))

In [1364]:
# all_data['engine_capacity_cc'] = all_data['engine_capacity_cc'].astype('str').str.replace('-1.0', '0.0')

In [1365]:
# all_data.to_csv('processed_data_2020.csv', index=False)

In [1366]:
from kmodes.kmodes import KModes
import numpy as np
import pickle


In [1368]:
columnsdrop = ['accident_reference', 'casualty_reference', 'vehicle_reference', 'casualty_severity', 'casualty_severity_name']
# model = KModes(n_jobs = -1, n_clusters = 5, init = 'Huang').fit(all_data.drop(columns=columnsdrop))

In [1384]:
# pickle.dump(model, open('Visualization/py/model.pkl', 'wb'))

In [1370]:
all_data_after = all_data[:]
all_data_after['Cluster Labels'] = model.labels_

C:\Users\20172010\AppData\Local\Temp\ipykernel_1360\311156270.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [1372]:
all_data_after.columns

Index(['accident_reference', 'casualty_severity', 'vehicle_reference',
       'casualty_reference', 'sex_of_casualty', 'age_band_of_casualty',
       'casualty_home_area_type', 'vehicle_type', 'journey_purpose_of_driver',
       'engine_capacity_cc', 'propulsion_code', 'age_of_vehicle',
       'generic_make_model', 'casualty_severity_name', 'Cluster Labels'],
      dtype='object')

In [1373]:
cluster_counts = pd.DataFrame()

for v in all_data_after.columns:
    if v in ['accident_reference', 'casualty_reference', 'vehicle_reference', 'casualty_severity', 'Cluster Labels']:
        continue
    column = v
    if 'severity' in v:
        norm = False
    else:
        norm = True
    cluster_unstack = all_data_after.groupby('Cluster Labels')[column].value_counts(normalize=norm).unstack(fill_value=0)
    columns = []
    for c, i in zip(cluster_unstack.columns, range(len(cluster_unstack.columns))):
        columns.append(column+'+'+str(c))

    cluster_unstack.columns = columns
    cluster_unstack = cluster_unstack.T
    cluster_counts = cluster_counts.append(cluster_unstack)

cluster_counts = cluster_counts.T
cluster_counts['count'] = all_data_after.groupby('Cluster Labels')['Cluster Labels'].count()


In [1374]:
exp = 2
cluster_counts['risk_score'] = all_data_after.groupby('Cluster Labels')['casualty_severity'].value_counts().unstack(fill_value=0).apply(lambda x: (x[1] * 1**exp) + (x[2] * 2**exp) + (x[3] * 3**exp), axis=1)

In [1379]:
import plotly.express as px
fig = px.bar(cluster_counts, x=cluster_counts.index, y='risk_score')
fig.show()

In [1387]:
# cluster_counts.to_csv('Visualization/data/profiles.csv')

In [1382]:
boolean = []
for c in cluster_counts.columns:
    if 'severity_name+Slight' in c:
        boolean.append(True)
    else:
        boolean.append(False)


In [1383]:
import plotly.express as px

long_df = px.data.medals_long()

fig = px.bar(cluster_counts.T[boolean])
fig.show()

In [1388]:
model = pickle.load(open('Visualization/py/model.pkl', 'rb'))

In [1389]:
centroids = model.cluster_centroids_
pd.DataFrame(centroids)

,0,1,2,3,4,5,6,7,8
0,Female,26 - 35,Urban area,Car,Normal,low,Petrol,new,FORD
1,Male,26 - 35,Urban area,Car,Normal,low,Petrol,middle-age,FORD
2,Male,26 - 35,Urban area,Car,Normal,low,Heavy oil,old,BMW
3,Male,46 - 55,Urban area,Car,Normal,low,Petrol,new,VAUXHALL
4,Male,46 - 55,Rural,Car,Normal,low,Petrol,new,Other


In [1390]:
show = []
for c in cluster_counts.columns:
    for v in ['age']:
        if v in c:
            show.append(c)

import plotly.express as px
df = px.data.iris()
fig = px.parallel_coordinates(cluster_counts, color=cluster_counts.index,
                              dimensions=show)
fig.show()

In [1391]:
from itertools import product

In [1393]:
a_data = all_data.drop(columns=columnsdrop)
uniques = [a_data[i].unique().tolist() for i in a_data.columns ]
prep = pd.DataFrame(product(*uniques), columns = a_data.columns)

In [1395]:
predictions = model.predict(prep)

In [1397]:
prep['profile'] = predictions

In [1401]:
# prep.to_csv('Visualization/data/predictions.csv', index=False)